In [16]:
import numpy as np

In [8]:
import requests
url = "https://raw.githubusercontent.com/rasbt/machine-learning-book/main/ch15/1268-0.txt"
text = requests.get(url)

In [9]:
text = text.text
type(text)

str

In [26]:
import numpy as np

In [27]:
import requests
url = "https://raw.githubusercontent.com/rasbt/machine-learning-book/main/ch15/1268-0.txt"
response = requests.get(url)
text = response.text

In [28]:
start_indx = text.find('THE MYSTERIOUS ISLAND')
end_indx = text.find('End of the Project Gutenberg')

text = text[start_indx: end_indx]
char_set = set(text)

print('total length:', len(text))
print("Unique characters:", len(char_set))

total length: 1135474
Unique characters: 81


In [29]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i, ch in enumerate(chars_sorted)}

char_array = np.array(chars_sorted)

text_encoded = np.array([char2int[ch] for ch in text], dtype=np.int32)

In [30]:
seq_length = 40

chunk_size = seq_length + 1

text_chunks = [text_encoded[i:i+chunk_size] for i in range(len(text_encoded)-chunk_size)]

In [31]:
import torch
from torch.utils.data import Dataset

class TextDataset(Dataset):
    def __init__(self, text_chunks):
        self.text_chunks = text_chunks
    
    def __len__(self):
        return len(self.text_chunks)
    
    def __getitem__(self, idx):
        text_chunk = self.text_chunks[idx]
        return text_chunk[:-1].long(), text_chunk[1:].long()

seq_dataset = TextDataset(torch.tensor(text_chunks))

In [34]:
device = torch.device("cuda:0")
# device = 'cpu'

In [35]:
from torch.utils.data import DataLoader
 
batch_size = 64

torch.manual_seed(1)
seq_dl = DataLoader(seq_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [36]:
xb, yb = next(iter(seq_dl))
xb.shape, yb.shape

(torch.Size([64, 40]), torch.Size([64, 40]))

# Building a character-level RNN model

In [37]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, rnn_hidden_size):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim) 
        self.rnn_hidden_size = rnn_hidden_size
        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, 
                           batch_first=True)
        self.fc = nn.Linear(rnn_hidden_size, vocab_size)

    def forward(self, x, hidden, cell):
        out = self.embedding(x).unsqueeze(1)
        out, (hidden, cell) = self.rnn(out, (hidden, cell))
        out = self.fc(out).reshape(out.size(0), -1)
        return out, hidden, cell

    def init_hidden(self, batch_size):
        hidden = torch.zeros(1, batch_size, self.rnn_hidden_size)
        cell = torch.zeros(1, batch_size, self.rnn_hidden_size)
        return hidden.to(device), cell.to(device)
    
vocab_size = len(char_array)
embed_dim = 256
rnn_hidden_size = 512

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size) 
model = model.to(device)
model

RNN(
  (embedding): Embedding(81, 256)
  (rnn): LSTM(256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=81, bias=True)
)

In [38]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10000 

torch.manual_seed(1)

for epoch in range(num_epochs):
    hidden, cell = model.init_hidden(batch_size)
    seq_batch, target_batch = next(iter(seq_dl))
    seq_batch = seq_batch.to(device)
    target_batch = target_batch.to(device)
    optimizer.zero_grad()
    loss = 0
    for c in range(seq_length):
        pred, hidden, cell = model(seq_batch[:, c], hidden, cell) 
        loss += loss_fn(pred, target_batch[:, c])
    loss.backward()
    optimizer.step()
    loss = loss.item()/seq_length
    if epoch % 500 == 0:
        print(f'Epoch {epoch} loss: {loss:.4f}')

Epoch 0 loss: 4.4052
Epoch 500 loss: 1.4586
Epoch 1000 loss: 1.3211
Epoch 1500 loss: 1.2773
Epoch 2000 loss: 1.2323
Epoch 2500 loss: 1.2424
Epoch 3000 loss: 1.1700
Epoch 3500 loss: 1.1341
Epoch 4000 loss: 1.0937
Epoch 4500 loss: 1.1501
Epoch 5000 loss: 1.0490
Epoch 5500 loss: 1.1494
Epoch 6000 loss: 1.1046
Epoch 6500 loss: 1.0246
Epoch 7000 loss: 1.0404
Epoch 7500 loss: 1.0399
Epoch 8000 loss: 1.0128
Epoch 8500 loss: 1.0241
Epoch 9000 loss: 0.9922
Epoch 9500 loss: 0.9787
